## Анализ последствий наводнений

In [1]:
import os, rasterio, torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms  import v2
from tqdm import tqdm
from typing import List, Optional
from rasterio.windows import Window
import segmentation_models_pytorch as smp
from sklearn.metrics import f1_score

In [2]:
# получить плитки с перекрытием

def get_tiles_with_overlap(image_width: int, image_height: int, 
                           tile_size: int, overlap: int) -> List[Window]:
    """
    Вычислите окна для плиток с заданным перекрытием по всему изображению

    Parameters:
        image_width (int): Ширина входного изображения в пикселях.
        image_height (int): Высота входного изображения в пикселях.
        tile_size (int): Размер каждой плитки (предполагается квадратная плитка).
        overlap (int): Количество перекрывающихся пикселей между соседними плитками.

    Returns:
        List[Window]: Список объектов rasterio Window, представляющих каждую плитку.
    """
    step_size = tile_size - overlap
    tiles = []
    for y in range(0, image_height, step_size):
        for x in range(0, image_width, step_size):
            window = Window(x, y, tile_size, tile_size)
            # Отрегулируйте окно, если оно выходит за границы изображения
            window = window.intersection(Window(0, 0, image_width, image_height))
            tiles.append(window)
    return tiles

def save_tile(src_dataset: rasterio.io.DatasetReader, window: Window, 
              output_folder: str, tile_index: int, image_id: int) -> None:
    """
    Извлечение и сохранение одной плитки из исходного набора данных.

    Parameters:
        src_dataset (rasterio.io.DatasetReader): Открытый rasterio-датасет (входное изображение).
        window (Window): Окно (rasterio Window object) определяющее плитку.
        output_folder (str): Папка, в которую будут сохранены плитки.
        tile_index (int): Индекс плитки, которая будет использоваться для именования файла.
        image_id (int): Идентификатор изображения, который будет использоваться для именования файла.

    Returns:
        None
    """
    transform = src_dataset.window_transform(window)
    tile_data = src_dataset.read(window=window)
    
    profile = src_dataset.profile
    profile.update({
        'driver': 'GTiff',
        'height': window.height,
        'width': window.width,
        'transform': transform
    })
    
    output_filename = os.path.join(output_folder, f"tile_{image_id}_{tile_index}.tif")
    with rasterio.open(output_filename, 'w', **profile) as dst:
        dst.write(tile_data)

def split_image(image_path: str, output_folder: str, mask_path: Optional[str] = None, 
                tile_size: int = 512, overlap: int = 128, image_id: int = 0) -> None:
    """
    Разделить большое изображение GeoTIFF и соответствующую маску (если она предусмотрена) на тайлы с перекрытием и сохраните их.

    Parameters:
        image_path (str): Путь к файлу входного изображения TIFF.
        mask_path (Optional[str]): Путь к соответствующей изображению маске в формате TIFF. Если None, обрабатывается только изображение.
        output_folder (str): Папка, в которую будут сохранены плитки.
        tile_size (int, optional): Размер плитки. По умолчанию512x512.
        overlap (int, optional): Количество пикселей, которые должны перекрываться между плитками. По умолчанию 128 pixels.
        image_id (int, optional): Идентификатор входного изображения, который будет использоваться для именования файла. 
            Defaults to 0.

    Returns:
        None
    """
    with rasterio.open(image_path) as src_image:
        image_width = src_image.width
        image_height = src_image.height

        # Создайте выходные каталоги для изображений и масок (если они есть).
        images_folder = os.path.join(output_folder, 'images')
        os.makedirs(images_folder, exist_ok=True)

        if mask_path:
            masks_folder = os.path.join(output_folder, 'masks')
            os.makedirs(masks_folder, exist_ok=True)

        # Получить список плиток с перекрытием
        tiles = get_tiles_with_overlap(image_width, image_height, tile_size, overlap)

        # Сохраните плитки изображения (и плитки маски, если они предусмотрены)
        if mask_path:
            with rasterio.open(mask_path) as src_mask:
                for idx, window in tqdm(enumerate(tiles)):
                    save_tile(src_image, window, images_folder, idx, image_id)
                    save_tile(src_mask, window, masks_folder, idx, image_id)
        else:
            for idx, window in tqdm(enumerate(tiles)):
                save_tile(src_image, window, images_folder, idx, image_id)


output_folder = 'data/' 

for i, name in enumerate(['1.tif', '2.tif', '4.tif', '5.tif']):#, '6_1.tif', '9_1.tif']):
    for over in [32, 64, 96]:
        split_image(
            image_path=f'train/images/{name}', mask_path=f'train/masks/{name}',
            output_folder=output_folder, tile_size=256,
            overlap=32, image_id=i*100+over
            )

In [3]:
# Получение списков данных для обучения и валидации

def get_data_list(img_path):
    """
    Retrieves a list of file names from the given directory.
    """
    name = []
    for _, _, filenames in os.walk(img_path): # given a directory iterates over the files
        for filename in filenames:
            f = filename.split('.')[0]
            name.append(f)

    df =  pd.DataFrame({'id': name}, index = np.arange(0, len(name))
                       ).sort_values('id').reset_index(drop=True)
    df = df['id'].values

    return np.delete(df, 0)

gdl = get_data_list('data/images/')

train_list, val_list = random_split(gdl, [.95, .05], generator=torch.Generator().manual_seed(42))

In [4]:
tr_len, val_len = len(train_list), len(val_list)

In [5]:
def image_padding(image, target_size=256):
    """
    Pad an image to a target size using reflection padding.
    """
    height, width = image.shape[1:3]
    pad_height = max(0, target_size - height)
    pad_width = max(0, target_size - width)
    padded_image = np.pad(image, ((0, 0), (0, pad_height),
                                  (0, pad_width)), mode='reflect')
    return padded_image


def mask_padding(mask, target_size=256):
    """
    Pad a mask to a target size using reflection padding.
    """
    height, width = mask.shape
    pad_height = max(0, target_size - height)
    pad_width = max(0, target_size - width)
    padded_mask = np.pad(mask, ((0, pad_height), (0, pad_width)),
                         mode='reflect')
    return padded_mask

augs = v2.Compose([v2.RandomHorizontalFlip(.4), 
                   v2.RandomVerticalFlip(.3),
                   v2.RandomRotation(degrees=90),
                   ])

class WaterDataset(Dataset):
    def __init__(self, img_path, mask_path, file_names, aug=False):
        self.img_path = img_path
        self.mask_path = mask_path
        self.file_names = file_names
        self.aug = aug

    def __len__(self):
            return len(self.file_names)

    def __getitem__(self, idx):
        with rasterio.open(self.img_path + self.file_names[idx] + '.tif') as fin:
            image = fin.read()
        # Добавим нормализацию
        image = v2.Normalize(mean=[0.]*10, std=[1.]*10)(image)
        image = image_padding(image).astype(np.float32)

        with rasterio.open(self.mask_path + self.file_names[idx] + '.tif') as fin:
            mask = fin.read(1)
        mask = mask_padding(mask)

        # Добавление расширения данных
        if self.aug:
            image, mask = augs(image, mask)

        return image, mask
    
train_ds = WaterDataset(img_path='data/images/', mask_path='data/masks/',
                        file_names=train_list, aug=True
                        )

val_ds = WaterDataset(img_path='data/images/', mask_path='data/masks/',
                      file_names=val_list)

bs = 32
train = DataLoader(train_ds, batch_size = bs, shuffle = True)
val = DataLoader(val_ds, batch_size = bs)

In [6]:
# Выбор модели и параметров обучения

model = smp.DeepLabV3Plus(encoder_name="timm-regnety_064", in_channels=10, classes=2)

model.segmentation_head[0] = torch.nn.Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
model.segmentation_head[2] = torch.nn.Sigmoid()

model = model.to('cuda')

lr, dt, l1, l0, ne = 9e-4, .99, 1e9, 1e9, 25
optimizer = torch.optim.AdamW(params = model.parameters(), lr = lr)

In [7]:
# Обучение модели

for epoch in range(ne):
    print('Эпоха №', epoch + 1)
    model.train()
    tr_loss, tr_f1 = 0, 0
    for i, batch in enumerate(tqdm(train)):
        ims, gts = batch[0].to('cuda'), batch[1].to('cuda')
        preds = model(ims)
        loss = torch.nn.functional.binary_cross_entropy(preds, gts.unsqueeze(1).to(dtype=torch.float32))
        tr_loss += loss.item()
        tr_f1 += f1_score(torch.round(preds.view(-1)).int().to('cpu'),
                          gts.view(-1).to('cpu'), average='macro')
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (i+1)%10 == 0:
            lr *= dt
            optimizer.param_groups[0]["lr"] = lr
    model.eval()
    with torch.no_grad():
        val_loss, val_f1 = 0, 0
        for ims, gts in tqdm (val):
            ims, gts = ims.to('cuda'), gts.to('cuda')
            preds = model(ims)
            val_loss += torch.nn.functional.binary_cross_entropy(preds, 
                                                                 gts.unsqueeze(1).to(dtype=torch.float32))
            val_f1 += f1_score(torch.round(preds.view(-1)).int().to('cpu'),
                               gts.view(-1).to('cpu'), average='macro')
    tr_loss /= tr_len
    tr_f1 /= tr_len
    val_loss /= tr_len
    val_f1 /= val_len
    print(f'lr: {lr:.9f} \t train: {tr_loss:.8f}, {tr_f1:.8f}\t val: {val_loss:.8f}, {val_f1:.8f}')
    if val_loss > l0 and l1 > l0: break
    elif epoch > 1 and val_loss < l1 < l0:
        torch.save(model, f"best_model_0.pt")
    l1, l0 = loss, l1

Эпоха № 1


100%|██████████| 16/16 [00:31<00:00,  1.98s/it]


lr: 0.000672455 	 train: 0.00025956, 0.02839847	 val: 0.00001182, 0.02899331
Эпоха № 2


100%|██████████| 16/16 [00:33<00:00,  2.12s/it]


lr: 0.000502440 	 train: 0.00025428, 0.02847202	 val: 0.00001145, 0.02898498
Эпоха № 3


...

100%|██████████| 16/16 [00:32<00:00,  2.03s/it]


lr: 0.000117000 	 train: 0.00018043, 0.02893837	 val: 0.00000880, 0.02945227
Эпоха № 8


100%|██████████| 16/16 [00:36<00:00,  2.29s/it]


lr: 0.000087419 	 train: 0.00017274, 0.02910861	 val: 0.00000869, 0.02958956
Эпоха № 9


 90%|████████▉ | 267/298 [11:52<01:23,  2.70s/it]

In [8]:
# Оценка сегментации

def mIoU(pred, gt):
    with torch.no_grad():
        pred, gt = pred.contiguous().view(-1), gt.contiguous().view(-1)
        iou_per_class = []
        for c in range(2):
            match_pred = pred == c
            match_gt   = gt == c
            if match_gt.long().sum().item() == 0:
                # iou_per_class.append(-1)
                iou_per_class.append(np.nan)
            else:
                intersect = torch.logical_and(match_pred, match_gt).sum().float().item()
                union = torch.logical_or(match_pred, match_gt).sum().float().item()
                iou = (intersect + 1e-10) / (union + 1e-10)
                iou_per_class.append(iou)        
    return np.nanmean(iou_per_class)

mIoU(torch.round(model(ims)).int(), gts)

0.8986936666625109

## Анализ последствий наводнений

**Описание датасета**

папка train:
- images мультиспектральные снимки большие
- masks соотвествующие им бинарные маски
- example_preds это пример предсказаний, чтобы можно было метрики прогнать
- osm карта OpenStreetMap для кусочка на котором будет считаться бизнес метрика

дальше подсчет метрик:
- calculate_metrics.py - принимает пути до файлов и считает много раз взвешенное
1. на вход получает две папки с предиктами и масками и сранивает macro_f1_score
2. на вход получает маску и предикт для двух снимков до и после затопления, а также osm для этой территории - на основе этих данных, вычисляется количество затопленных домов. Далее вычисляется macro_f1_score для ситуации до и отдельно для ситуации после, а результаты усредняются.
3. Усредняется результат первого и второго пункта
- run_calculation.sh - скрипт для запуска py файла, как пример

helper.py - много разных вспомогательных функций. 
- Визуализация
- Деление большого снимка на тайлы, чтобы собрать датасет нужного патча
- Даталодер с паддингами для мультиспектральных данных на pytorch
- Примерный пайплайн, как можно сохранить результаты работы модели


**Предлагается разработать модель для сегментации водной поверхности на основе мультиспектральных данных, а также провести количественную оценку затопленных объектов инфраструктуры.**

***На выходе нужно, чтобы `pipline` прочитал большое изображение, разделил его на части, сделал вывод масок, а затем объединил результат в большую маску и сохранил в geoTiff***

**У нас будут Sentinel-2A снимки с 10 каналами:**

| Name | Description                                          | Resolution |
|------|------------------------------------------------------|------------|
| B02  | Blue, 492.4 nm (S2A), 492.1 nm (S2B)                 | 10m        |
| B03  | Green, 559.8 nm (S2A), 559.0 nm (S2B)                | 10m        |
| B04  | Red, 664.6 nm (S2A), 665.0 nm (S2B)                  | 10m        |
| B05  | Vegetation red edge, 704.1 nm (S2A), 703.8 nm (S2B)  | 20m        |
| B06  | Vegetation red edge, 740.5 nm (S2A), 739.1 nm (S2B)  | 20m        |
| B07  | Vegetation red edge, 782.8 nm (S2A), 779.7 nm (S2B)  | 20m        |
| B08  | NIR, 832.8 nm (S2A), 833.0 nm (S2B)                  | 10m        | 
| B8A  | Narrow NIR, 864.7 nm (S2A), 864.0 nm (S2B)           | 20m        |
| B11  | SWIR, 1613.7 nm (S2A), 1610.4 nm (S2B)               | 20m        |
| B12  | SWIR, 2202.4 nm (S2A), 2185.7 nm (S2B)               | 20m        |

**И маски от 0 до 1:**

0. backgound
1. water

Задача заключалось в разработке модели для сегментации водной поверхности - получить изображение, где вода - белая область, остальная поверхность - черная.  Решение проводилось на основе исследования мультиспектральных данных - спутниковых снимков по 10 каналам: видимого диапазона, индексов растительности и разных областей инфракрасного диапазона. Количественная оценка затопленных объектов инфраструктуры проводиться на основе полученного черно-белого изображения - маски. Полученное решение должно прочитать большое изображение, разделил его на части, сделал вывод масок, а затем объединил результат в большую маску, сохранил ее в формате geoTiff.
Особенностью полученного решения является использование глубоких нейронных сетей: 
- современные архитектура и кодировщик изображений
- адаптация исходной архитектуры нейронной сети под цели задачи: 10 входных каналов и 1 выходной
. Особенности обучения модели:
- разделение исходных данных (10-ти канальных "изображений") на части с различным перекрытием;
- дальнейшее расширение набора обучаемых данных на основе отражений и поворотов;
- изменение шага обучения модели
- контроль переобучения модели